### List modules

In [2]:
import numpy as np
import os
import matplotlib
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline 
import warnings
import json
import glob
import pickle
warnings.filterwarnings('ignore')

### Import Data

In [5]:
with open("variables/train_size", 'rb') as f:
    train_size = pickle.load(f)
with open("variables/test_size", 'rb') as f:
    test_size = pickle.load(f)  
print("Train samples: ", train_size)
print("Test samples: ", test_size)

Train samples:  401
Test samples:  400


### File extensions and count

In [9]:
with open("variables/train_list", 'rb') as f:
    train_list = pickle.load(f)
with open("variables/ext_dict", 'rb') as f:
    ext_dict = pickle.load(f)
 
print("Extensions: ", ext_dict)      


for file_ext in ext_dict:
    print("Files with extension ",file_ext, ": ", len([file for file in train_list if  file.endswith(file_ext)]))

Extensions:  ['mp4', 'json']
Files with extension  mp4 :  400
Files with extension  json :  1


# Checking json file
<img src="files/labels.png">

#### ^^ All missing data has in original is associated with the label REAL (Real videos are the originals)

# Plot data distribution
<img src="files/plot data.png">

In [18]:
!pip install facenet-pytorch

In [20]:
class DetectionPipeline:

    def __init__(self, detector, n_frames=None, batch_size=60, resize=None):

        self.detector = detector
        self.n_frames = n_frames
        self.batch_size = batch_size
        self.resize = resize
        
            
    def __call__(self, filename):
 
   
        v_cap = cv2.VideoCapture(filename)
        v_len = int(v_cap.get(cv2.CAP_PROP_FRAME_COUNT))
        print(v_len)
    
        if self.n_frames is None:
            sample = np.arange(0, v_len)
        else:
            sample = np.linspace(0, v_len - 1, self.n_frames).astype(int)

   
        faces = []
        frames = []
        for j in range(v_len):
            success = v_cap.grab()
            if j in sample:
    
                success, frame = v_cap.retrieve()
                if not success:
                    continue
                frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                frame = Image.fromarray(frame)
                
    
                if self.resize is not None:
                    frame = frame.resize([int(d * self.resize) for d in frame.size])
                frames.append(frame)

   
                if len(frames) % self.batch_size == 0 or j == sample[-1]:
                    faces.extend(self.detector(frames))
                    frames = []

        v_cap.release()
        
        return faces


In [21]:
import copy
def process_faces(faces, resnet):

    faces = [f for f in faces if f is not None]
    if(len(faces) == 0):
        return []

    faces = torch.cat(faces).to(device)
    if(len(faces)<290):
        return []

    faces = faces[:290]

    embeddings = resnet(faces)

    centroid = embeddings.mean(dim=0)
    
    x = (embeddings - centroid).norm(dim=1).cpu().numpy()
    
    return x


In [45]:
#Code for extracting and processing faces from videos




#detection_pipeline = DetectionPipeline(detector=mtcnn, batch_size=60, resize=0.25)
#import json

#with open('/content/drive/Shared drives/deepfake/deepfake-detection-challenge/train_sample_videos/metadata.json') as f:
#  data = json.load(f)

#filenames = glob.glob('/content/drive/Shared drives/deepfake/deepfake-detection-challenge/train_sample_videos/*.mp4')
#total_files = len(filenames)

#X = []
#y = []
#start = time.time()
#n_processed = 0

#with torch.no_grad():
#  for i, filename in tqdm(enumerate(filenames), total=len(filenames)):
#    print(i, filename)
#    faces = detection_pipeline(filename)
#    z = process_faces(faces, resnet)
#    if len(z)!=0:
#      X.append(z)
#      if data[filename[87:]]['label'] == 'FAKE':
#        y.append(1)
#      else:
#        y.append(0)
#    n_processed += len(faces)

# Evaluating accuracy

In [24]:
with open("variables/casiaX", 'rb') as f:
    casiaX = pickle.load(f)
with open("variables/casiaY", 'rb') as f:
    casiaY = pickle.load(f)

In [31]:
!pip install -U scikit-learn

In [37]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
from sklearn import svm
from sklearn.metrics import accuracy_score

X_train, X_test, y_train, y_test = train_test_split(casiaX, casiaY, test_size=0.85, random_state=5)

# Using CASIA-Webface

In [34]:
clf = LogisticRegression(random_state=0).fit(X_train, y_train)
y_pred_lr = clf.predict(X_test)
print("Logisitic Regression: ", accuracy_score(y_test,y_pred_lr))

Logisitic Regression:  0.8012820512820513


In [35]:
gnb = GaussianNB()
y_pred_gnb = gnb.fit(X_train, y_train).predict(X_test)
print("GaussianNB:", accuracy_score(y_test,y_pred_gnb))

GaussianNB: 0.6410256410256411


In [38]:
clf = svm.SVC(gamma='auto')
clf.fit(X_train, y_train)
y_pred_svm = clf.predict(X_test)
print("SVM: ",accuracy_score(y_test,y_pred_svm))

SVM:  0.8076923076923077


# 

In [41]:
#using VGGFace2
with open("variables/VGGX", 'rb') as f:
    X1 = pickle.load(f)
with open("variables/VGGY", 'rb') as f:
    y1 = pickle.load(f)

In [42]:
X_train, X_test, y_train, y_test = train_test_split(X1, y1, test_size=0.85, random_state=5)

clf = LogisticRegression(random_state=0).fit(X_train, y_train)
y_pred_lr = clf.predict(X_test)
print("Logisitic Regression: ", accuracy_score(y_test,y_pred_lr))

Logisitic Regression:  0.8076923076923077


In [43]:
gnb = GaussianNB()
y_pred_gnb = gnb.fit(X_train, y_train).predict(X_test)
print("GaussianNB:", accuracy_score(y_test,y_pred_gnb))

GaussianNB: 0.6826923076923077


In [44]:
clf = svm.SVC(gamma='auto')
clf.fit(X_train, y_train)
y_pred_svm = clf.predict(X_test)
print("SVM: ",accuracy_score(y_test,y_pred_svm))

SVM:  0.8076923076923077
